Importing a CSV file into a postgres database

Steps

1.Import the CSV file into a pandas df

In [5]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [6]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 72CC-C2F3

 Directory of c:\Users\BERNARD ESHUN\Desktop\DOCUMENTS\DATA ANALYTICS\PYTHON\Data_Science

02/23/2024  08:44 PM    <DIR>          .
02/23/2024  08:44 PM    <DIR>          ..
02/23/2024  08:41 PM        46,306,270 coviddeaths.csv
12/25/2023  01:27 PM        33,923,346 CovidDeaths.xlsx
12/25/2023  01:25 PM        39,431,079 CovidVaccinations.xlsx
02/23/2024  08:51 PM            24,428 Files_to_postgresql.ipynb
               4 File(s)    119,685,123 bytes
               2 Dir(s)  105,696,063,488 bytes free


In [7]:
#Importing files into pandas

df = pd.read_excel("CovidDeaths.xlsx")
df.head()

,iso_code,continent,location,date,population,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2020-01-03,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,41128772,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Clean table name
#Lower case letters
#remove all white spaces and $
#replace -,/,\\ with _

file = "CovidDeaths."
clean_table_name = file.lower().replace(" ","_").replace("?","") \
    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
        .replace(")","").replace(r"(","").replace("$","").replace(".","") \
        
        

clean_table_name

'coviddeaths'

In [9]:
#Clean header name
#Lower case letters
#remove all white spaces and $
#replace -,/,\\ with _

df.columns = [x.lower().replace(" ","_").replace("?","") \
    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
        .replace(")","").replace(r"(","").replace("$","").replace(".","") for x in df.columns]


df.columns

Index(['iso_code', 'continent', 'location', 'date', 'population',
       'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths',
       'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million'],
      dtype='object')

In [10]:
#Creating  our database tables in postgres

create table coviddeaths
(
iso_code                                      varchar,
continent                                     varchar,
location                                      varchar,
date                                          date,
population                                     bigint,
total_cases                                  float,
new_cases                                    float,
new_cases_smoothed                           float,
total_deaths                                 float,
new_deaths                                   float,
new_deaths_smoothed                          float,
total_cases_per_million                      float,
new_cases_per_million                        float,
new_cases_smoothed_per_million               float,
total_deaths_per_million                     float,
new_deaths_per_million                       float,
new_deaths_smoothed_per_million              float,
reproduction_rate                            float,
icu_patients                                 float,
icu_patients_per_million                     float,
hosp_patients                                float,
hosp_patients_per_million                    float,
weekly_icu_admissions                        float,
weekly_icu_admissions_per_million            float,
weekly_hosp_admissions                       float,
weekly_hosp_admissions_per_million           float,
);



SyntaxError: invalid syntax (2796403676.py, line 3)

In [11]:
#Checking my data types

df.dtypes 



iso_code                                      object
continent                                     object
location                                      object
date                                  datetime64[ns]
population                                     int64
total_cases                                  float64
new_cases                                    float64
new_cases_smoothed                           float64
total_deaths                                 float64
new_deaths                                   float64
new_deaths_smoothed                          float64
total_cases_per_million                      float64
new_cases_per_million                        float64
new_cases_smoothed_per_million               float64
total_deaths_per_million                     float64
new_deaths_per_million                       float64
new_deaths_smoothed_per_million              float64
reproduction_rate                            float64
icu_patients                                 f

In [12]:
#Created a dictionary to replace pandas datatypes with postgres datatypes

replacements = { 
    'object': 'varchar',
    'float64': 'float',
    'int64': 'bigint',
    'timedelta64[ns]':'varchar',
    'datetime64':'timestamp',
    'bool': 'boolean',
    'json_normalize': 'json',  
    'datetime64[ns]': 'date',
    'datetime64': 'time'
    }

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'bigint',
 'timedelta64[ns]': 'varchar',
 'datetime64': 'time',
 'bool': 'boolean',
 'json_normalize': 'json',
 'datetime64[ns]': 'date'}

In [13]:
#joining the database tables their accepted postgres datatypes

col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))


col_str

'iso_code varchar, continent varchar, location varchar, date date, population bigint, total_cases float, new_cases float, new_cases_smoothed float, total_deaths float, new_deaths float, new_deaths_smoothed float, total_cases_per_million float, new_cases_per_million float, new_cases_smoothed_per_million float, total_deaths_per_million float, new_deaths_per_million float, new_deaths_smoothed_per_million float, reproduction_rate float, icu_patients float, icu_patients_per_million float, hosp_patients float, hosp_patients_per_million float, weekly_icu_admissions float, weekly_icu_admissions_per_million float, weekly_hosp_admissions float, weekly_hosp_admissions_per_million float'

In [14]:
#connecting to the postgres database
conn_string =hostname = 'localhost'
database = 'projectsdb'
username = 'postgres'
pwd = 'password'
port_id = 5432
conn = psycopg2.connect(
    host=hostname,
    dbname=database,
    user=username,
    password=pwd,
    port=port_id)
cursor = conn.cursor()
print('opened the database successfully')







opened the database successfully


In [15]:
#dropping tables with same name

cursor.execute("drop table if exists coviddeaths;")



In [16]:
#create the table

cursor.execute("CREATE TABLE coviddeaths "
               "(iso_code VARCHAR, continent VARCHAR, location VARCHAR, "
               "date DATE, population BIGINT, total_cases FLOAT, new_cases FLOAT, new_cases_smoothed FLOAT, "
               "total_deaths FLOAT, new_deaths FLOAT, new_deaths_smoothed FLOAT, total_cases_per_million FLOAT, "
               "new_cases_per_million FLOAT, new_cases_smoothed_per_million FLOAT, total_deaths_per_million FLOAT, "
               "new_deaths_per_million FLOAT, new_deaths_smoothed_per_million FLOAT, reproduction_rate FLOAT, "
               "icu_patients FLOAT, icu_patients_per_million FLOAT, hosp_patients FLOAT, "
               "hosp_patients_per_million FLOAT, weekly_icu_admissions FLOAT, "
               "weekly_icu_admissions_per_million FLOAT, weekly_hosp_admissions FLOAT, "
               "weekly_hosp_admissions_per_million FLOAT)")



In [17]:
#inserting values into the tables

#save df to CSV 
df.to_csv('coviddeaths.csv',header=df.columns,index=False,encoding='utf-8')


#open the CSV file,save it as an object 

my_file = open('coviddeaths.csv')
print('file opened in memory')


file opened in memory


In [18]:
#upload the object to db

SQL_STATEMENT = """
COPY coviddeaths FROM STDIN WITH 
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql=SQL_STATEMENT,file=my_file)
print('file copied to db')

file copied to db


In [19]:
#Granting user access

cursor.execute("grant select on table coviddeaths to public")
conn.commit()

cursor.close()
print('coviddeaths tables imported to postgres Successfully')

coviddeaths tables imported to postgres Successfully
